<center><img src="https://github.com/hse-ds/iad-applied-ds/blob/master/2021/hw/hw1/img/logo_hse.png?raw=1" width="1000"></center>

<h1><center>Прикладные задачи анализа данных</center></h1>
<h2><center>Домашнее задание 4: рекомендательные системы</center></h2>

# Введение

В этом задании Вы продолжите работать с данными из семинара [Articles Sharing and Reading from CI&T Deskdrop](https://www.kaggle.com/gspmoreira/articles-sharing-reading-from-cit-deskdrop).

# Загрузка и предобработка данных

In [ ]:
import pandas as pd
import numpy as np
import math
import scipy.sparse as sps

Загрузим данные и проведем предобраотку данных как на семинаре.

In [ ]:
! pip install --user kaggle

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 70 bytes


In [ ]:
!kaggle datasets download -d gspmoreira/articles-sharing-reading-from-cit-deskdrop
!unzip articles-sharing-reading-from-cit-deskdrop.zip -d articles

  0% 0.00/8.20M [00:00<?, ?B/s]
100% 8.20M/8.20M [00:00<00:00, 256MB/s]
Archive:  articles-sharing-reading-from-cit-deskdrop.zip
  inflating: articles/shared_articles.csv  
  inflating: articles/users_interactions.csv  


In [ ]:
articles_df = pd.read_csv("articles/shared_articles.csv")
articles_df = articles_df[articles_df["eventType"] == "CONTENT SHARED"]
articles_df.head(2)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en


In [ ]:
interactions_df = pd.read_csv("articles/users_interactions.csv")
interactions_df.head(2)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US


In [ ]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

In [ ]:
# зададим словарь определяющий силу взаимодействия
event_type_strength = {
   "VIEW": 1.0,
   "LIKE": 2.0, 
   "BOOKMARK": 2.5, 
   "FOLLOW": 3.0,
   "COMMENT CREATED": 4.0,  
}

interactions_df["eventStrength"] = interactions_df.eventType.apply(lambda x: event_type_strength[x])

Оставляем только тех пользователей, которые произамодействовали более чем с пятью статьями.

In [ ]:
users_interactions_count_df = (
    interactions_df
    .groupby(["personId", "contentId"])
    .first()
    .reset_index()
    .groupby("personId").size())
print("# users:", len(users_interactions_count_df))

users_with_enough_interactions_df = \
    users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[["personId"]]
print("# users with at least 5 interactions:",len(users_with_enough_interactions_df))

# users: 1895
# users with at least 5 interactions: 1140


Оставляем только те взаимодействия, которые относятся к отфильтрованным пользователям.

In [ ]:
interactions_from_selected_users_df = interactions_df.loc[np.in1d(interactions_df.personId,
            users_with_enough_interactions_df)]

In [ ]:
print(f"# interactions before: {interactions_df.shape}")
print(f"# interactions after: {interactions_from_selected_users_df.shape}")

# interactions before: (72312, 9)
# interactions after: (69868, 9)


Объединяем все взаимодействия пользователя по каждой статье и сглажиываем полученный результат, взяв от него логарифм.

In [ ]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = (
    interactions_from_selected_users_df
    .groupby(["personId", "contentId"]).eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index().set_index(["personId", "contentId"])
)
interactions_full_df["last_timestamp"] = (
    interactions_from_selected_users_df
    .groupby(["personId", "contentId"])["timestamp"].last()
)
        
interactions_full_df = interactions_full_df.reset_index()
interactions_full_df.head(5)

,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.000000,1470395911
1,-1007001694607905623,-6623581327558800021,1.000000,1487240080
2,-1007001694607905623,-793729620925729327,1.000000,1472834892
3,-1007001694607905623,1469580151036142903,1.000000,1487240062
4,-1007001694607905623,7270966256391553686,1.584963,1485994324


Разобьём выборку на обучение и контроль по времени.

In [ ]:
from sklearn.model_selection import train_test_split

split_ts = 1475519530
interactions_train_df = interactions_full_df.loc[interactions_full_df.last_timestamp < split_ts].copy()
interactions_test_df = interactions_full_df.loc[interactions_full_df.last_timestamp >= split_ts].copy()

print(f"# interactions on Train set: {len(interactions_train_df)}")
print(f"# interactions on Test set: {len(interactions_test_df)}")

interactions_train_df

# interactions on Train set: 29329
# interactions on Test set: 9777


,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.0,1470395911
2,-1007001694607905623,-793729620925729327,1.0,1472834892
6,-1032019229384696495,-1006791494035379303,1.0,1469129122
7,-1032019229384696495,-1039912738963181810,1.0,1459376415
8,-1032019229384696495,-1081723567492738167,2.0,1464054093
...,...,...,...,...
39099,997469202936578234,9112765177685685246,2.0,1472479493
39100,998688566268269815,-1255189867397298842,1.0,1474567164
39101,998688566268269815,-401664538366009049,1.0,1474567449
39103,998688566268269815,6881796783400625893,1.0,1474567675


Для удобства подсчёта качества запишем данные в формате, где строка соответствует пользователю, а столбцы будут истинными метками и предсказаниями в виде списков.

In [ ]:
interactions = (
    interactions_train_df
    .groupby("personId")["contentId"].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={"contentId": "true_train"})
    .set_index("personId")
)

interactions["true_test"] = (
    interactions_test_df
    .groupby("personId")["contentId"].agg(lambda x: list(x))
)

# заполнение пропусков пустыми списками
interactions.loc[pd.isnull(interactions.true_test), "true_test"] = [
    "" for x in range(len(interactions.loc[pd.isnull(interactions.true_test), "true_test"]))]

interactions.head(1)

,true_train,true_test
personId,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72..."


# Библиотека LightFM

Для рекомендации Вы будете пользоваться библиотекой [LightFM](https://making.lyst.com/lightfm/docs/home.html), в которой реализованы популярные алгоритмы. Для оценивания качества рекомендации, как и на семинаре, будем пользоваться метрикой *precision@10*.

In [ ]:
!pip install lightfm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 310 kB 7.5 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705384 sha256=c0bce19b6dd6f2ed8b9cad416e20b3d55fb0047b20759ceebd53bdd42080ffcf
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a1021914
Successfully built lightfm


In [ ]:
from lightfm import LightFM 
from lightfm.evaluation import precision_at_k 

## Задание 1 (2 балла)

Модели в LightFM работают с разреженными матрицами. Создайте разреженные матрицы `data_train` и `data_test` (размером количество пользователей на количество статей), такие что на пересечении строки пользователя и столбца статьи стоит сила их взаимодействия, если взаимодействие было, и стоит ноль, если взаимодействия не было.

In [ ]:
# Ваш код здесь

data_train = pd.pivot_table(
    interactions_train_df,
    values='eventStrength',
    index='personId',
    columns='contentId').fillna(0)#.astype('Sparse') # .astype(pd.SparseDtype("float64",0))

data_test = pd.pivot_table(
    interactions_test_df,
    values='eventStrength',
    index='personId',
    columns='contentId').fillna(0)#.astype('Sparse')

In [ ]:
data_train.shape[0] == len(pd.unique(interactions_train_df.personId))

True

In [ ]:
data_train.shape[1] == len(pd.unique(interactions_train_df.contentId))

True

In [ ]:
# получим из pivot_table обычный датафрейм

df_train = pd.DataFrame(data_train.reset_index().values[:, 1:],
                        index=data_train.index.values,
                        columns=data_train.columns.values,)
                        # dtype=pd.SparseDtype("float64",0)) # .astype(pd.SparseDtype(int, fill_value=0))

df_test = pd.DataFrame(data_test.reset_index().values[:, 1:],
                       index=data_test.index.values,
                       columns=data_test.columns.values,)
                      #  dtype=pd.SparseDtype("float64",0))

In [ ]:
# сохраним массив со всеми пользователями
all_persons = pd.unique(interactions_full_df.personId)
# сохраним массив тех пользователями, которые не вошли в трейн
train_persons = np.setdiff1d(all_persons, df_train.index.values)
# сохраним массив тех пользователями, которые не вошли в тест
test_persons = np.setdiff1d(all_persons, df_test.index.values)

# повторим для контента
all_contents = pd.unique(interactions_full_df.contentId)
train_contents = np.setdiff1d(all_contents, df_train.columns.values) 
test_contents = np.setdiff1d(all_contents, df_test.columns.values)

In [ ]:
train_ddf = pd.DataFrame(0, index=train_persons, columns=train_contents)
train_full = pd.concat([df_train, train_ddf], ignore_index=True).fillna(0)
train_full = train_full.astype(pd.SparseDtype("float64",0))

test_ddf = pd.DataFrame(0, index=test_persons, columns=test_contents)
test_full = pd.concat([df_test, test_ddf], ignore_index=True).fillna(0)
test_full = test_full.astype(pd.SparseDtype("float64",0))

## Задание 2 (1 балл)

Обучите модель LightFM с `loss="warp"` и посчитайте *precision@10* на тесте.

In [ ]:
# Ваш код здесь

model = LightFM(loss='warp')
model.fit(train_full.sparse.to_coo().tocsr())

test_precision = precision_at_k(model, test_full.sparse.to_coo().tocsr(), k=10).mean()
test_precision 

0.0056008147

## Задание 3 (3 балла)

При вызове метода `fit` LightFM позволяет передавать в `item_features` признаковое описание объектов. Воспользуемся этим. Будем получать признаковое описание из текста статьи в виде [TF-IDF](https://ru.wikipedia.org/wiki/TF-IDF) (можно воспользоваться `TfidfVectorizer` из scikit-learn). Создайте матрицу `feat` размером количесвто статей на размер признакового описание и обучите LightFM с `loss="warp"` и посчитайте precision@10 на тесте.

In [ ]:
# Ваш код здесь
# сохраним отдельно все имеющиеся тексты для статей (всех статей из обучающей и тестовой выборок)
articles_texts = articles_df[articles_df.contentId.isin(all_contents)][['contentId', 'text']]

# найдем айди статей, чьих текстов у нас нет
contents8 = np.setdiff1d(all_contents, articles_texts.contentId.values)

# сохраним датефрейм с текстами для всех текстов
# будем ставить пустую строку в соответствие статьям, для которых нет текста
texts = pd.concat([articles_texts,
                   pd.DataFrame({
                       'contentId': contents8,
                       'text': ['']*len(contents8)
                       })
                   ])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MaxAbsScaler

tfidf_vectorizer = TfidfVectorizer()
# bag of words
feat = tfidf_vectorizer.fit_transform(texts.text)

# нормируем данные
scaler = MaxAbsScaler()
feat = scaler.fit_transform(feat)

In [ ]:
feat.shape

(2984, 71875)

In [ ]:
len(all_contents)

2984

In [ ]:
model2 = LightFM(loss='warp')
model2.fit(train_full.sparse.to_coo().tocsr(), item_features=feat)

test_precision_tfidf = precision_at_k(model2, test_full.sparse.to_coo().tocsr(), item_features=feat, k=10).mean()
test_precision_tfidf 

0.006415479

## Задание 4 (2 балла)

В задании 3 мы использовали сырой текст статей. В этом задании необходимо сначала сделать предобработку текста (привести к нижнему регистру, убрать стоп слова, привести слова к номральной форме и т.д.), после чего обучите модель и оценить качество на тестовых данных.

In [ ]:
# Ваш код здесь
from string import punctuation

# очистим текст от спецсимволов и знаков препинания
texts.text = texts.text.apply(lambda x: x.translate(str.maketrans('', '', punctuation)))

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

# проведем стемминг, при этом приведем все слова к нижнему регистру, уберем стоп слова
stemmer = SnowballStemmer('english')

def MyTokenizer(themes):

  stop = stopwords.words('english')
  themes_tokenized = [' '.join([stemmer.stem(w) for w in word_tokenize(theme.lower()) if w.isalpha() and 
                               w not in stop]) for theme in themes]
  
  return themes_tokenized

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
texts.text = MyTokenizer(texts.text)

In [ ]:
feat2 = tfidf_vectorizer.fit_transform(texts.text)

scaler = MaxAbsScaler()
feat2 = scaler.fit_transform(feat2)

In [ ]:
model3 = LightFM(loss='warp')
model3.fit(train_full.sparse.to_coo().tocsr(), item_features=feat2)

test_precision_tfidf2 = precision_at_k(model3, test_full.sparse.to_coo().tocsr(), item_features=feat2, k=10).mean()
test_precision_tfidf2 

0.0016293279

Улучшилось ли качество предсказания?

Нет :)

## Задание 5 (2 балла)

Подберите гиперпараметры модели LightFM (`n_components` и др.) для улучшения качества модели.

In [ ]:
# Ваш код здесь
model4 = LightFM(k=10, loss='warp')
model4.fit(train_full.sparse.to_coo().tocsr(), item_features=feat2, epochs=10)

test_precision_final = precision_at_k(model4, test_full.sparse.to_coo().tocsr(), item_features=feat2, k=10).mean()
test_precision_final 

0.005397149

In [ ]:
model5 = LightFM(no_components=30, k=10, loss='warp')
model5.fit(train_full.sparse.to_coo().tocsr(), item_features=feat2, epochs=10)

test_precision_final = precision_at_k(model5, test_full.sparse.to_coo().tocsr(), item_features=feat2, k=10).mean()
test_precision_final 

0.0027494908

In [ ]:
model6 = LightFM(no_components=30, k=10, loss='warp')
model6.fit(train_full.sparse.to_coo().tocsr(), item_features=feat2, epochs=30)

test_precision_final = precision_at_k(model6, test_full.sparse.to_coo().tocsr(), item_features=feat2, k=10).mean()
test_precision_final 

0.0034623218

In [ ]:
model7 = LightFM(k=10, loss='warp')
model7.fit(train_full.sparse.to_coo().tocsr(), item_features=feat2, epochs=100)

test_precision_final = precision_at_k(model7, test_full.sparse.to_coo().tocsr(), item_features=feat2, k=10).mean()
test_precision_final 

0.0046843183

In [ ]:
model8 = LightFM(k=10, loss='warp', learning_rate=0.06)
model8.fit(train_full.sparse.to_coo().tocsr(), item_features=feat2, epochs=100)

test_precision_final = precision_at_k(model8, test_full.sparse.to_coo().tocsr(), item_features=feat2, k=10).mean()
test_precision_final 

0.005397149

In [ ]:
model9 = LightFM(k=10, loss='warp', learning_rate=0.06)
model9.fit(train_full.sparse.to_coo().tocsr(), item_features=feat2, epochs=10)

test_precision_final = precision_at_k(model9, test_full.sparse.to_coo().tocsr(), item_features=feat2, k=10).mean()
test_precision_final

0.0038696537

In [ ]:
model10 = LightFM(k=10, loss='warp', learning_rate=0.04)
model10.fit(train_full.sparse.to_coo().tocsr(), item_features=feat2, epochs=10)

test_precision_final = precision_at_k(model10, test_full.sparse.to_coo().tocsr(), item_features=feat2, k=10).mean()
test_precision_final

0.0026476579

Итак, я обучила несколько моделей, меняя в каждой новой какой-то из имеющихся параметров или добавляя новый. Как видно из значений лоссов выше, качество то повышалось, то снова падало от модели к модели. 

Тем не менее мне все же удалось его улучшить, по сравнению с отметкой, достигнутой в задании 4. Качество обученной модели (model8) равнялось 0.005397149 при следующих значениях параметров:

* k=10, 
* learning_rate=0.06

При этом модель обучалась на протяжении 100 эпох.

Будет справедливо сказать, что точно такое же качество было достигнуто при добавлении (со своим значением) в изначальную модель всего одного параметра: k=10 (model4).

Вывод: качество отстойное как ни крути :)

## Бонусное задание (3 балла)

Выше мы использовали достаточно простое представление текста статьи в виде TF-IDF. В этом задании Вам нужно представить текст статьи (можно вместе с заголовком) в виде эмбеддинга полученного с помощью рекуррентной сети или трансформера (можно использовать любую предобученную модель, которая Вам нравится). Обучите модель с ипользованием этих эмеддингов и сравните результаты с предыдущими.

In [ ]:
# Ваш код здесь